## Cleaning

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly.express as px
import nbformat
from wordcloud import WordCloud
from datetime import datetime
from geopy.distance import geodesic

In [4]:
# load raw data
raw_df = pd.read_csv("./raw/biennale_submission_data_raw.csv")
countries_df = pd.read_csv("./geo/countries.csv")

In [5]:
# rename columns
raw_df.columns = raw_df.columns.str.lower().str.replace(' ', '_').str.replace(':', '')
raw_df.rename(columns={'parent_item_[h]': 'parent_item'}, inplace=True)
raw_df.rename(columns={'hasparent_[h]': 'hasparent'}, inplace=True)
raw_df.rename(columns={'late_(spontaneous)_flag': 'late_spontaneous_flag'}, inplace=True)
raw_df.rename(columns={'project_dimensions_(square_meters)': 'project_dimensions_sqm'}, inplace=True)
raw_df.rename(columns={'z_projects_-_tracking': 'z_projects_tracking'}, inplace=True)
raw_df.rename(columns={'z_claire_&_gabriel_selection': 'z_claire_gabriel_selection'}, inplace=True)
raw_df.rename(columns={'stage_2_project_title_': 'stage_2_project_title'}, inplace=True)
raw_df.rename(columns={'stage_3--_exhibition_section': 'stage_3_exhibition_section'}, inplace=True)
raw_df.rename(columns={'stage_3--_preferred_exhibition_strategy': 'stage_3_preferred_exhibition_strategy'}, inplace=True)
raw_df.rename(columns={'project_area_of_focus_': 'project_area_of_focus'}, inplace=True)

# lowercase keywords
raw_df["relevant_exhibition_section"] = raw_df["relevant_exhibition_section"].str.lower()
raw_df["personal_perspective"] = raw_df["personal_perspective"].str.lower()
raw_df["stage_2_exhibition_section"] = raw_df["stage_2_exhibition_section"].str.lower()
raw_df["stage_2_exhibition_section"] = raw_df["stage_2_exhibition_section"].str.lower()
raw_df["geography_of_residence"] = raw_df["geography_of_residence"].str.lower()
raw_df["primary_keywords"] = raw_df["primary_keywords"].str.lower()

# clean values
raw_df['personal_perspective'] = raw_df['personal_perspective'].replace({'transnational (immigrant, refugee, multicultural) perspective': 'transnational/immigrant/refugee/multicultural perspective'})
raw_df['invited_as'] = raw_df['invited_as'].replace(['NO', '[do not send an invitation or rejection]'], np.nan)


In [6]:
# convert to dictionary for fast lookups
country_to_lat = dict(zip(countries_df["name"].str.lower(), countries_df["lat"]))
country_to_long = dict(zip(countries_df["name"].str.lower(), countries_df["lon"]))
# lookup lat/lon
raw_df["geo_lat"] = raw_df["geography_of_residence"].map(country_to_lat)
raw_df["geo_lon"] = raw_df["geography_of_residence"].map(country_to_long)
# distance
venice = {"lat": 45.438759, "lon":12.327145}
raw_df["distance_km"] = raw_df.apply(lambda row: 
    geodesic((row['geo_lat'], row['geo_lon']), (venice["lat"], venice["lon"])).km 
    if not any(pd.isna([row['geo_lat'], row['geo_lon'], venice["lat"], venice["lon"]])) 
    else np.nan, axis=1)
# remove duplicates
raw_df = raw_df.drop_duplicates()

## Exports

### Keywords and Geography

In [7]:
keyword_geo = raw_df[["primary_keywords","geography_of_residence","geo_lat","geo_lon"]].dropna().reset_index()

# explode keywords into separate rows
keyword_geo['primary_keywords'] = keyword_geo['primary_keywords'].str.split(',')  # Split on ", "
keyword_geo = keyword_geo.explode('primary_keywords')  # Expand into multiple rows
# trim whitespace
keyword_geo['primary_keywords'] = keyword_geo['primary_keywords'].str.strip().reset_index(drop=True)

# group and count
keyword_geo_grp = keyword_geo.groupby(['primary_keywords', 'geography_of_residence']).size().reset_index(name='count')

# geocode values
keyword_geo_grp["geo_lat"] = keyword_geo_grp["geography_of_residence"].map(country_to_lat)
keyword_geo_grp["geo_lon"] = keyword_geo_grp["geography_of_residence"].map(country_to_long)

# save files
# keyword_geo.to_csv("./subset/keyword_geo.csv")
# keyword_geo_grp.to_csv("./subset/keyword_geo_grp.csv")

### Distance

In [8]:
# distance
distance = raw_df[["proposal_id","project_title","invited_as","geography_of_residence","distance_km"]].dropna().reset_index()
# distance.to_csv("./subset/distance.csv")

### Keywords and Perspectives

In [10]:
keyword_perspective = raw_df[["primary_keywords","personal_perspective",]].dropna().reset_index()
# explode keywords into separate rows
keyword_perspective['primary_keywords'] = keyword_perspective['primary_keywords'].str.split(',')  # Split on ", "
keyword_perspective = keyword_perspective.explode('primary_keywords')  # Expand into multiple rows
# trim whitespace
keyword_perspective['primary_keywords'] = keyword_perspective['primary_keywords'].str.strip().reset_index(drop=True)

# re-sort rows
keyword_perspective = keyword_perspective[["personal_perspective","primary_keywords"]].dropna().reset_index()
# explode personal perspective into separate rows
keyword_perspective['personal_perspective'] = keyword_perspective['personal_perspective'].str.split(',')  # Split on ", "
keyword_perspective = keyword_perspective.explode('personal_perspective')  # Expand into multiple rows
# trim whitespace
keyword_perspective['personal_perspective'] = keyword_perspective['personal_perspective'].str.strip().reset_index(drop=True)

# group and count
keyword_perspective_grp = keyword_perspective.groupby(['personal_perspective', 'primary_keywords']).size().reset_index(name='count')
perspective_keyword_grp = keyword_perspective.groupby(['primary_keywords', 'personal_perspective']).size().reset_index(name='count')

# save files
# keyword_perspective.to_csv("./subset/keyword_perspective.csv")
# keyword_perspective_grp.to_csv("./subset/keyword_perspective_grp.csv")
# perspective_keyword_grp.to_csv("./subset/perspective_keyword_grp.csv")

### Affiliation 

This is a rough guess based on text substrings. We could use someone to manually categorize these for an accurate account.

In [11]:
# affiliaiton
affiliaiton = raw_df[["proposal_id","project_title","primary_affiliation","primary_affiliation_link","primary_keywords",'personal_perspective','invited_as']].reset_index()
affiliaiton.loc[affiliaiton['primary_affiliation_link'].str.contains('(?i).edu', na=False), 'affiliation_type'] = 'education'
affiliaiton.loc[affiliaiton['primary_affiliation'].str.contains('(?i)univers|insti|school|politecnico|laboratory', na=False), 'affiliation_type'] = 'education'
affiliaiton.loc[affiliaiton['primary_affiliation'].str.contains('(?i)independent|indi', na=False), 'affiliation_type'] = 'independent'
affiliaiton.loc[affiliaiton['primary_affiliation'].str.contains('(?i)studio|archi|atelier|firm', na=False), 'affiliation_type'] = 'studio'

# save files
# affiliaiton.to_csv("./subset/affiliaiton.csv")

### Project Type

In [189]:
# project type
# raw_df["project_type"]

### Selections

In [190]:
# id, stage_1, stage_2, stage_3, 

# selections
# raw_df["stage_3_placement"].value_counts()

# X, Y, Z, Material Bank, Event
#raw_df["invited_as"].value_counts()

# raw_df["review_status"].value_counts()

# raw_df[raw_df["review_status"].str.contains("Duplicate", case=False, na=False)]

# need to explode on comma
# raw_df["relevant_exhibition_section"].value_counts()